In [106]:
import pathlib
import zipfile
import os
import tensorflow as tf
from PIL import Image as Image
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import random
from keras import regularizers
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [107]:
dataset_path = "../DatasetL2/Project_Dataset_Unzip/Dataset"

In [108]:
image_h = 75
image_w = 75
batch_s = 32

In [109]:
def load_data(data_path, test_size=0.2):
    train = []
    test = []
    for file in os.listdir(data_path):
        random_number = random.random()
        image = Image.open(data_path + "\\" + file)
        pix = np.array(image.getdata()).reshape(image.size[0], image.size[1], 3)
        if random_number < test_size :
            test.append(pix)
        else:
            train.append(pix)
    return train, test

x_train, x_test = load_data(dataset_path)
print(len(x_train))
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=len(x_train))

x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=len(x_test))

#x_train_noisy = np.clip(x_train_noisy, 0., 1.)
#x_test_noisy = np.clip(x_test_noisy, 0., 1.)


123


d:\document\ecole\cesi\annee 5\projet_datascience_a5\venv\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
d:\document\ecole\cesi\annee 5\projet_datascience_a5\venv\lib\site-packages\ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [110]:
input_img = keras.Input(shape=(image_h, image_w, 1))

encoded = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
encoded = keras.layers.MaxPooling2D((2, 2), padding='same')(encoded)
encoded = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
encoded = keras.layers.MaxPooling2D((2, 2), padding='same')(encoded)

decoded = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
decoded = keras.layers.UpSampling2D((2, 2))(decoded)
decoded = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(decoded)
decoded = keras.layers.UpSampling2D((2, 2))(decoded)
#decoded = keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoded)

auto_encoder = keras.Model(input_img, decoded)
auto_encoder.compile(optimizer = 'adam',
                  loss = 'mean_absolute_error')
auto_encoder.fit(x_train_noisy, x_train,
                epochs=10,
                batch_size=batch_s,
                shuffle=True,
                validation_data=(x_test_noisy, x_test)
                )
auto_encoder.summary()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
tf.keras.utils.plot_model(auto_encoder, show_shapes=True)

In [ ]:
decoded_imgs = auto_encoder.predict(x_test_noisy, verbose=1)
print(decoded_imgs.shape)

In [ ]:
print(decoded_imgs[1].shape)

In [ ]:
def display_image(X, n):
    plt.figure(figsize=(20, 2))
    for i in range(n):
        ax = plt.subplot(1,n,i+1)
        plt.imshow(X[i].reshape(image_h, image_w))
        #plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
display_image(decoded_imgs,10)